In [2]:
import pandas as pd
import unicodedata
import re
from sklearn.utils import shuffle

ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')


response_df = pd.read_csv('./ANLI_R3_Survey_Final_September 4, 2020_09.04.csv')
response_attention_checks = pd.DataFrame({
    'responseID': response_df['Random ID'],
    'check1': response_df['Q302'],
    'check2': response_df['Q303'],
    'check3': response_df['Q304'],
    'check4': response_df['Q305'],
    'check5': response_df['Q306'],
    'check6': response_df['Q307'],
    'check7': response_df['Q308'],
    'check8': response_df['Q309'],
    'check9': response_df['Q310'],
    'check10': response_df['Q311'],
    'check11': response_df['Q312'],
    'check12': response_df['Q313']

})
response_questions = response_df.drop(['StartDate','EndDate','Status','IPAddress','Progress','Duration (in seconds)','Finished','RecordedDate','ResponseId','RecipientLastName', \
'RecipientFirstName','RecipientEmail','ExternalReference','LocationLatitude','LocationLongitude','DistributionChannel','UserLanguage'],axis=1)
# response_questions.set_index('Random ID',inplace=True)
response_questions.rename(columns={"Random ID": "RandomId"}, inplace=True)
response_questions

,QID2,QID3,QID4,QID5,QID6,QID7,QID8,QID9,QID10,QID11,...,QID298,QID299,QID300,Q308,Q309,Q310,Q311,Q312,Q313,RandomId
0,"Given the Premise, is the Hypothesis definitel...","Given the Premise, is the Hypothesis definitel...","Given the Premise, is the Hypothesis definitel...","Given the Premise, is the Hypothesis definitel...","Given the Premise, is the Hypothesis definitel...","Given the Premise, is the Hypothesis definitel...","Given the Premise, is the Hypothesis definitel...","Given the Premise, is the Hypothesis definitel...","Given the Premise, is the Hypothesis definitel...","Given the Premise, is the Hypothesis definitel...",...,"Given the Premise, is the Hypothesis definitel...","Given the Premise, is the Hypothesis definitel...","Given the Premise, is the Hypothesis definitel...","Given the Premise, is the Hypothesis definitel...","Given the Premise, is the Hypothesis definitel...","Given the Premise, is the Hypothesis definitel...","Given the Premise, is the Hypothesis definitel...","Given the Premise, is the Hypothesis definitel...","Given the Premise, is the Hypothesis definitel...",Random ID
1,"{""ImportId"":""QID2""}","{""ImportId"":""QID3""}","{""ImportId"":""QID4""}","{""ImportId"":""QID5""}","{""ImportId"":""QID6""}","{""ImportId"":""QID7""}","{""ImportId"":""QID8""}","{""ImportId"":""QID9""}","{""ImportId"":""QID10""}","{""ImportId"":""QID11""}",...,"{""ImportId"":""QID298""}","{""ImportId"":""QID299""}","{""ImportId"":""QID300""}","{""ImportId"":""QID308""}","{""ImportId"":""QID309""}","{""ImportId"":""QID310""}","{""ImportId"":""QID311""}","{""ImportId"":""QID312""}","{""ImportId"":""QID313""}","{""ImportId"":""Random ID""}"
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,neither definitely correct nor definitely inco...,NaN,NaN,NaN,NaN,61445326288230
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,definitely incorrect,NaN,NaN,NaN,NaN,NaN,35876488802023
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38444590615108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,definitely incorrect,NaN,NaN,NaN,NaN,NaN,12559431069530
86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither definitely correct nor definitely inco...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,definitely correct,21957201068289
87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,definitely correct,NaN,NaN,NaN,NaN,NaN,94022159604355
88,neither definitely correct nor definitely inco...,NaN,NaN,NaN,NaN,NaN,NaN,definitely correct,NaN,NaN,...,NaN,NaN,NaN,NaN,neither definitely correct nor definitely inco...,NaN,NaN,NaN,NaN,89232312422245


In [ ]:
response_questions.iloc[2]

In [54]:
# check for participants who failed both attention checks
failed_attention_checks = []
attention_answer_map = {
    'check1': 'definitely incorrect',
    'check2': 'definitely correct',
    'check3': 'neither definitely correct nor definitely incorrect',
    'check4': 'definitely correct',
    'check5': 'definitely incorrect',
    'check6': 'neither definitely correct nor definitely incorrect',
    'check7': 'definitely incorrect',
    'check8': 'neither definitely correct nor definitely incorrect',
    'check9': 'definitely correct',
    'check10': 'neither definitely correct nor definitely incorrect',
    'check11': 'definitely incorrect',
    'check12': 'definitely correct'
}
outfile = open("passed_attention_checks.txt",'w+')
for id in range(2,len(response_attention_checks)):
    response = response_attention_checks.iloc[id]
    attention_correct_count = 0
    for k,v in attention_answer_map.items():
        if response[k] == v:
            attention_correct_count +=  1
        
    if attention_correct_count == 2:
        failed_attention_checks.append((response['responseID'],id))
        outfile.write(response['responseID'])
        outfile.write('\n')

print(len(failed_attention_checks))
print(failed_attention_checks)
outfile.close()

37
[('61445326288230', 2), ('5918052908964', 5), ('85769710945896', 8), ('23467815760523', 11), ('59494899632409', 12), ('58872996177524', 14), ('43165576714091', 15), ('68321020714938', 17), ('92963079409673', 21), ('90455432306043', 23), ('57974082743749', 24), ('94496444007381', 26), ('95626433915458', 27), ('6383683509193', 30), ('69377727154642', 37), ('2982167620211', 39), ('13732979469932', 44), ('3813853557221', 48), ('48222230374813', 50), ('18138183839619', 54), ('78198984265327', 60), ('37883877265267', 62), ('8198794443160', 63), ('95119172590784', 64), ('95733413449488', 65), ('25947750336490', 66), ('37273650430142', 67), ('69503748207353', 71), ('13361342321150', 72), ('46358098718337', 77), ('81421692809090', 78), ('53600934171117', 79), ('80719325295649', 82), ('29851378197781', 83), ('12559431069530', 85), ('89232312422245', 88), ('7078240392729', 89)]


In [ ]:
import math
none_answered = []
for id in failed_attention_checks:
    response = response_attention_checks.iloc[id[1]]
    not_answered = 0
    for k,v in attention_answer_map.items():
        if isinstance(response[k], str) == True:
            not_answered += 1
    if not_answered != 2:
        none_answered.append(id)

print(len(none_answered))
print(none_answered)

In [ ]:
response_attention_checks.iloc[87]

In [ ]:
# check for participants who did not answer all questions
outfile = open("unanswered_questions.txt",'w+')
for id in range(2,len(response_questions)):
    null_counts = pd.notna(response_questions.iloc[id])
    count = null_counts.isin([True]).sum(axis=0)
    # check for 16 to account for random id
    if count != 16:
        print(f"{count} {id}")
        outfile.write(response_questions.iloc[id]['Random ID'])
        outfile.write("\n")

outfile.close()

In [ ]:
null_counts = pd.notna(response_questions.iloc[15])
print(null_counts)

# Strip out all the phase 1 rejections for phase 2. But also caculate the accuracy scores for the phase 1 rejects

In [38]:
response_questions

,QID2,QID3,QID4,QID5,QID6,QID7,QID8,QID9,QID10,QID11,...,QID298,QID299,QID300,Q308,Q309,Q310,Q311,Q312,Q313,RandomId
0,"Given the Premise, is the Hypothesis definitel...","Given the Premise, is the Hypothesis definitel...","Given the Premise, is the Hypothesis definitel...","Given the Premise, is the Hypothesis definitel...","Given the Premise, is the Hypothesis definitel...","Given the Premise, is the Hypothesis definitel...","Given the Premise, is the Hypothesis definitel...","Given the Premise, is the Hypothesis definitel...","Given the Premise, is the Hypothesis definitel...","Given the Premise, is the Hypothesis definitel...",...,"Given the Premise, is the Hypothesis definitel...","Given the Premise, is the Hypothesis definitel...","Given the Premise, is the Hypothesis definitel...","Given the Premise, is the Hypothesis definitel...","Given the Premise, is the Hypothesis definitel...","Given the Premise, is the Hypothesis definitel...","Given the Premise, is the Hypothesis definitel...","Given the Premise, is the Hypothesis definitel...","Given the Premise, is the Hypothesis definitel...",Random ID
1,"{""ImportId"":""QID2""}","{""ImportId"":""QID3""}","{""ImportId"":""QID4""}","{""ImportId"":""QID5""}","{""ImportId"":""QID6""}","{""ImportId"":""QID7""}","{""ImportId"":""QID8""}","{""ImportId"":""QID9""}","{""ImportId"":""QID10""}","{""ImportId"":""QID11""}",...,"{""ImportId"":""QID298""}","{""ImportId"":""QID299""}","{""ImportId"":""QID300""}","{""ImportId"":""QID308""}","{""ImportId"":""QID309""}","{""ImportId"":""QID310""}","{""ImportId"":""QID311""}","{""ImportId"":""QID312""}","{""ImportId"":""QID313""}","{""ImportId"":""Random ID""}"
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,neither definitely correct nor definitely inco...,NaN,NaN,NaN,NaN,61445326288230
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,definitely incorrect,NaN,NaN,NaN,NaN,NaN,35876488802023
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38444590615108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,definitely incorrect,NaN,NaN,NaN,NaN,NaN,12559431069530
86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither definitely correct nor definitely inco...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,definitely correct,21957201068289
87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,definitely correct,NaN,NaN,NaN,NaN,NaN,94022159604355
88,neither definitely correct nor definitely inco...,NaN,NaN,NaN,NaN,NaN,NaN,definitely correct,NaN,NaN,...,NaN,NaN,NaN,NaN,neither definitely correct nor definitely inco...,NaN,NaN,NaN,NaN,89232312422245


In [121]:
# failed_attention_checks = [f[1] for f in failed_attention_checks]
# response_questions = response_questions[response_questions.index not in failed_attention_checks]
drop_indexes = list(response_questions.index)
drop_indexes = [i for i in drop_indexes if i not in failed_attention_checks]
print(drop_indexes)
response_questions.drop(response_questions.index[drop_indexes],inplace=True)
response_questions
# print(response_questions)

[0, 1, 3, 4, 6, 7, 9, 10, 13, 16, 18, 19, 20, 22, 25, 28, 29, 31, 32, 33, 34, 35, 36, 38, 40, 41, 42, 43, 45, 46, 47, 49, 51, 52, 53, 55, 56, 57, 58, 59, 61, 68, 69, 70, 73, 74, 75, 76, 80, 81, 84, 86, 87]


,QID2,QID3,QID4,QID5,QID6,QID7,QID8,QID9,QID10,QID11,...,QID298,QID299,QID300,Q308,Q309,Q310,Q311,Q312,Q313,RandomId
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,neither definitely correct nor definitely inco...,NaN,NaN,NaN,NaN,61445326288230
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,definitely correct,definitely incorrect,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,neither definitely correct nor definitely inco...,NaN,NaN,5918052908964
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,neither definitely correct nor definitely inco...,NaN,NaN,85769710945896
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,neither definitely correct nor definitely inco...,NaN,NaN,NaN,NaN,23467815760523
12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,definitely incorrect,NaN,59494899632409
14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,definitely correct,NaN,NaN,NaN,58872996177524
15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,neither definitely correct nor definitely inco...,NaN,NaN,NaN,NaN,43165576714091
17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,neither definitely correct nor definitely inco...,NaN,NaN,NaN,NaN,68321020714938
21,NaN,NaN,definitely correct,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,definitely correct,92963079409673
23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,definitely correct,NaN,NaN,NaN,90455432306043


In [63]:
len(list(response_questions.index))

37

In [64]:
import json

answer_map = {
    'definitely correct': 'e',
    'definitely incorrect': 'c',
    'neither definitely correct nor definitely incorrect': 'n'
}


first_rejects = pd.read_csv("./phase_1_rejections.csv")
# response_questions.drop(response_questions.index[list(first_rejects.row)],inplace=True)
questions = open("./ANLI_R3_Survey_Final.qsf", 'r')
js = json.load(questions)
labels = pd.read_json('./anli/r3/train.jsonl',lines=True)
labels.drop(['reason','uid'],axis=1,inplace=True)

# use df to calculate human accuracys
human_performance = {
    'randomID':[],
    'numberCorrect':[],
}
i=0
for id in range(len(list(response_questions.index))):
    # print(id)
    answers = dict(response_questions.iloc[id].dropna())
    human_performance['randomID'].append(answers['RandomId'])
    human_performance['numberCorrect'].append(0)
    for k,v in answers.items():
        if k == 'RandomId':
            continue
        for j in range(21, 319):
            if js['SurveyElements'][j]['PrimaryAttribute'] == k:
                question = js['SurveyElements'][j]['Payload']['QuestionText']
                # get rid of the template
                question = question.replace("<strong>", "").replace("</strong>","").replace("""<span style="font-weight: bolder;">Given the Premise, is the Hypothesis definitely correct, definitely incorrect, or neither? You should only choose definitely correct if the Hypothesis is clearly stated in the Premise. You should only choose definitely incorrect if the Hypothesis is clearly contradicted</span><span style="font-weight: bolder;"> by the Premise.</span>
<div><br>""", "").replace("""<p><span style="font-weight: bolder;"><b>""", "").replace("""</b></span> <span style="white-space: pre-wrap;">""","").replace("</span></p>","").replace("<br><p><span><b>Hypothesis: </b></span>","").replace("</p>\n</div>","").replace("Premise: ", "")
                # separate the premise and hypothesis
                sentences = question.split("\n")
                # sentences = set(sentences)
                sentences = [z for z in sentences if z != '']
                # if id == len(response_questions) - 1:
                #     print(sentences)
                #     print('\n')
                
                # find the correct answer and increment numberCorrect if they got it right
                premise = sentences[0]
                hypothesis = sentences[1]
                # print(hypothesis)
                try:
                    correct_label = labels[labels['hypothesis'] == hypothesis]['label'].values[0]
                except IndexError:
                    print(f"'{hypothesis}'' not found")
                    human_performance['numberCorrect'][i] += 1
                    break
                # print(correct_label)
                if answer_map[v] == correct_label:
                    # print(i)
                    human_performance['numberCorrect'][i] += 1
                
                break
    i += 1


print(human_performance)
print(i)
print(len(human_performance['randomID']))


out_df = pd.DataFrame(human_performance)

out_df.to_csv('./results.csv')
            





'That is the kind of critic I dont like .'' not found
{'randomID': ['61445326288230', '5918052908964', '85769710945896', '23467815760523', '59494899632409', '58872996177524', '43165576714091', '68321020714938', '92963079409673', '90455432306043', '57974082743749', '94496444007381', '95626433915458', '6383683509193', '69377727154642', '2982167620211', '13732979469932', '3813853557221', '48222230374813', '18138183839619', '78198984265327', '37883877265267', '8198794443160', '95119172590784', '95733413449488', '25947750336490', '37273650430142', '69503748207353', '13361342321150', '46358098718337', '81421692809090', '53600934171117', '80719325295649', '29851378197781', '12559431069530', '89232312422245', '7078240392729'], 'numberCorrect': [6, 4, 8, 9, 7, 12, 12, 7, 10, 1, 10, 5, 5, 3, 12, 11, 11, 12, 10, 5, 3, 6, 9, 11, 6, 7, 5, 12, 10, 12, 7, 9, 11, 11, 7, 10, 8]}
37
37


In [18]:
answers = dict(response_questions.iloc[-1].dropna())
answers
# labels[labels['premise'] == 'Moreover, Canada encourages an expansion of religious freedom in China, including Tibet, and I do not doubt that Canada would welcome a dialogue between Chinese authorities and the Dalai Lama, a world spiritual leader who earned the Nobel Peace Prize in 1989 for his dedication to peace and human rights.' a labels['hypothesis'] == 'Canada which for more freedom of belief coming out of the other country ']
# labels[labels['premise'] == 'Moreover, Canada encourages an expansion of religious freedom in China, including Tibet, and I do not doubt that Canada would welcome a dialogue between Chinese authorities and the Dalai Lama, a world spiritual leader who earned the Nobel Peace Prize in 1989 for his dedication to peace and human rights.']
# labels[labels['premise'] == 'Moreover, Canada encourages an expansion of religious freedom in China, including Tibet, and I do not doubt that Canada would welcome a dialogue between Chinese authorities and the Dalai Lama, a world spiritual leader who earned the Nobel Peace Prize in 1989 for his dedication to peace and human rights.' and labels['hypothesis'] == 'Canada which for more freedom of belief coming out of the other country ']

# labels[labels['hypothesis'] == 'Canada which for more freedom of belief coming out of the other country ']['label'].values

# labels[labels.premise.str.contains(,case=False)]

{'QID24': 'neither definitely correct nor definitely incorrect',
 'QID46': 'definitely correct',
 'QID101': 'definitely correct',
 'QID102': 'neither definitely correct nor definitely incorrect',
 'QID107': 'definitely correct',
 'QID136': 'neither definitely correct nor definitely incorrect',
 'Q302': 'definitely incorrect',
 'QID193': 'neither definitely correct nor definitely incorrect',
 'QID198': 'definitely incorrect',
 'QID220': 'definitely correct',
 'QID264': 'neither definitely correct nor definitely incorrect',
 'QID282': 'definitely correct',
 'QID290': 'neither definitely correct nor definitely incorrect',
 'QID299': 'definitely correct',
 'Q309': 'neither definitely correct nor definitely incorrect',
 'RandomId': '7078240392729'}

# Calaculate fleiss kappa IAA

In [89]:
import numpy as np
test = np.array([[3,0,0],[0,3,0],[0,0,3]])
print(test)
fleiss_kappa(test)

[[3 0 0]
 [0 3 0]
 [0 0 3]]
3.0
9.0
[3 3 3]


1.0

In [126]:
import numpy as np
# entails is the first column, condractis the second, and neutral the third
answer_map = {
    'definitely correct': 0,
    'definitely incorrect': 1,
    'neither definitely correct nor definitely incorrect': 2
}


questions = open("./ANLI_R3_Survey_Final.qsf", 'r')
js = json.load(questions)
# response_df = pd.read_csv('./ANLI_R3_Survey_Final_September 4, 2020_09.04.csv')
# response_questions = response_df.drop(['StartDate','EndDate','Status','IPAddress','Progress','Duration (in seconds)','Finished','RecordedDate','ResponseId','RecipientLastName', \
# 'RecipientFirstName','RecipientEmail','ExternalReference','LocationLatitude','LocationLongitude','DistributionChannel','UserLanguage'],axis=1)
# response_questions.set_index('Random ID',inplace=True)
# response_questions.rename(columns={"Random ID": "RandomId"}, inplace=True)


M = np.zeros((298,3))

print(len(response_questions.index))
for id in range(len(list(response_questions.index))):
    # print(id)
    answers = dict(response_questions.iloc[id].dropna())
    for k,v in answers.items():
        if k == 'RandomId':
            continue
        i = 0
        for j in range(21, 319):
            if js['SurveyElements'][j]['PrimaryAttribute'] == k:
                M[i][answer_map[v]] += 1
                break
            i += 1
print(M)
print(fleiss_kappa(M))

37
[[0. 0. 1.]
 [0. 0. 0.]
 [2. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 0.]
 [3. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]
 [0. 0. 1.]
 [2. 0. 2.]
 [0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 2.]
 [1. 0. 0.]
 [1. 1. 0.]
 [1. 0. 0.]
 [2. 0. 0.]
 [0. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 1. 3.]
 [1. 0. 1.]
 [0. 2. 0.]
 [2. 0. 0.]
 [0. 0. 1.]
 [0. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 2. 0.]
 [0. 0. 1.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [2. 1. 0.]
 [1. 0. 1.]
 [1. 0. 0.]
 [1. 1. 1.]
 [1. 0. 0.]
 [0. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 2. 0.]
 [6. 0. 0.]
 [0. 0. 0.]
 [0. 0. 1.]
 [0. 0. 0.]
 [1. 0. 0.]
 [3. 1. 0.]
 [0. 2. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 1. 1.]
 [0. 0. 0.]
 [2. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]
 [2. 0. 0.]
 [3. 0. 0.]
 [0. 1. 0.]
 [0. 0. 2.]
 [1. 0. 0.]
 [0. 2. 1.]
 [0. 1. 1.]
 [1. 0. 0.]
 [0. 2. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 3.]
 [0. 0. 0.]
 [0. 1. 1.]
 [0. 2. 0.]
 [0. 1. 1.]
 [0. 2. 1.]
 [0. 2. 0.]
 [0. 0. 1.]
 [0. 2. 0.]
 [1. 1. 0.]
 

In [123]:
def fleiss_kappa(M):
    """Computes Fleiss' kappa for group of annotators.
    :param M: a matrix of shape (:attr:'N', :attr:'k') with 'N' = number of subjects and 'k' = the number of categories.
        'M[i, j]' represent the number of raters who assigned the 'i'th subject to the 'j'th category.
    :type: numpy matrix
    :rtype: float
    :return: Fleiss' kappa score
    """
    N, k = M.shape  # N is # of items, k is # of categories
    # n_annotators = float(np.sum(M[0, :]))  # # of annotators
    n_annotators = 37
    # tot_annotations = N * n_annotators  # the total # of annotations
    tot_annotations = 481
    print(n_annotators)
    print(tot_annotations)
    category_sum = np.sum(M, axis=0)  # the sum of each category over all items
    print(category_sum)

    # chance agreement
    p = category_sum / tot_annotations  # the distribution of each category over all annotations
    print(p)
    PbarE = np.sum(p * p)  # average chance agreement over all categories
    print(PbarE)

    # observed agreement
    P = (np.sum(M * M, axis=1) - n_annotators) / (n_annotators * (n_annotators - 1))
    # print(P)
    Pbar = np.sum(P) / N  # add all observed agreement chances per item and divide by amount of items
    print(Pbar)

    return round((Pbar - PbarE) / (1 - PbarE), 4)

# Drop the ones who get less than 33% accuracy including attention checks

In [4]:
to_drop = pd.read_csv("./rejections.csv")
to_drop.randomID

0     38444590615108
1     45814142562448
2     90455432306043
3      6236248067580
4     47813011775724
5     76947914436459
6     34203467890620
7     39416330726817
8     75171326450072
9     13275972031988
10    94022159604355
Name: randomID, dtype: int64

In [8]:
drop_indexes = list(to_drop.randomID)
drop_indexes
# drop_indexes = [i for i in drop_indexes if i not in failed_attention_checks]
# print(drop_indexes)
# response_questions.drop(response_questions.index[drop_indexes],inplace=True)
for i in range(len(response_questions)):
    if res_questions.iloc[i].randomID in drop_indexes:
        response_questions.drop(response_questions.iloc[i],inplace=True)
response_questions

NameError: name 'response_question' is not defined